In [21]:
import boto3
import json
import base64
import time
from botocore.config import Config
config = Config(
       connect_timeout=1000,
    read_timeout=1000,
)

session = boto3.session.Session(region_name='us-east-1')
client = session.client(service_name = 'bedrock-runtime', 
                                 config=config)


PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"

In [87]:
from json import JSONDecodeError
import re
def parse(pattern:str, text: str) -> str:
    match = re.search(pattern, text, re.DOTALL)
    if match:
        text = match.group(1)
        return text.strip()
    else:
        raise JSONDecodeError

## Canvas
- text 2 image

In [80]:
system_1 = \
"""
You are a Prompt Rewriting Expert for text-to-image models, with extensive knowledge in photography and panting. 
You specialize in helping users improve their text prompts according to specific rules to achieve better model outputs, sometimes modifying the original intent if necessary. 

##You excel in the following areas:##
Comprehensive understanding of the world, physical laws, and various interactive video scenarios
Rich imagination to visualize perfect, visually striking video scenes from simple prompts
Extensive film industry expertise as a master director, capable of enhancing simple video descriptions with optimal cinematography and visual effects

##Your prompt rewriting should follow these guidelines:##
- Prompting for image generation models differs from prompting for large language models (LLMs). Image generation models do not have the ability to reason or interpret explicit commands. Therefore, it's best to phrase your prompt as if it were an image caption rather than a command or conversation.You might want to include details about the subject, action, environment, lighting, style, and camera position.
- Consider adding modifiers like aspect ratios, image quality settings, or post-processing instructions to refine the output.
- Avoid topics such as pornography, racial discrimination, and toxic words.
- Be concise and less then 90 words.
- Do not use negation words like "no", "not", "without", and so on in your prompt. The model doesn't understand negation in a prompt and attempting to use negation will result in the opposite of what you intend. For example, a prompt such as "a fruit basket with no bananas" will actually signal the model to include bananas. Instead, you can use a negative prompt, via the negative prompt, to specify any objects or characteristics that you want to exclude from the image. For example "bananas".
- An effective prompt often includes short descriptions of...
1. the subject
2. the environment
3. (optional) the position or pose of the subject
4. (optional) lighting description
5. (optional) camera position/framing
6. (optional) the visual style or medium ("photo", "illustration", "painting", and so on)

##Good Examples##
- Prompt: realistic editorial photo of female teacher standing at a blackboard with a warm smile
- Negative Prompt: crossed arms

- Prompt: whimsical and ethereal soft-shaded story illustration: A woman in a large hat stands at the ship's railing looking out across the ocean
- Negative Prompt: clouds, waves

- Prompt: drone view of a dark river winding through a stark Iceland landscape, cinematic quality

- Prompt: A cool looking stylish man in an orange jacket, dark skin, wearing reflective glasses. Shot from slightly low angle, face and chest in view, aqua blue sleek building shapes in background.

##Ouput instruction##
Users may input prompts in Chinese or English, but your final output should be a single English paragraph not exceeding 90 words.
Put the prompt in <prompt></prompt>, and if has negative prompt, then put in <negative_prompt></negative_prompt>

"""

In [93]:
user_prompt = "吃冰棍的小女孩"

In [94]:
%%time
system = [
    {
        "text": system_1
    }
]

messages = [
    {
        "role": "user",
        "content": [
         {"text": f"Please optimize:{user_prompt}"},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"maxTokens": 2000, "topP": 0.9, "temperature": 0.8}


model_response = client.converse_stream(
    modelId=PRO_MODEL_ID, messages=messages, system=system, inferenceConfig=inf_params
)

text = ""
stream = model_response.get("stream")
if stream:
    for event in stream:
        if "contentBlockDelta" in event:
            text += event["contentBlockDelta"]["delta"]["text"]
            print(event["contentBlockDelta"]["delta"]["text"], end="")

<prompt>Young girl enjoying a popsicle on a sunny day, vibrant summer colors, close-up shot highlighting her joyful expression and the popsicle, soft background of a park with green foliage.</prompt>
<negative_prompt>overly bright colors, unnatural expressions</negative_prompt>CPU times: user 25 ms, sys: 52 µs, total: 25 ms
Wall time: 1.38 s


In [95]:
generated_prompt = parse(pattern = r"<prompt>(.*?)</prompt>",text=text)
print(len(generated_prompt))
print(generated_prompt)

182
Young girl enjoying a popsicle on a sunny day, vibrant summer colors, close-up shot highlighting her joyful expression and the popsicle, soft background of a park with green foliage.


In [96]:
negative_prompt = parse(pattern = r"<negative_prompt>(.*?)</negative_prompt>",text=text)
print(len(negative_prompt))
print(negative_prompt)

43
overly bright colors, unnatural expressions


In [91]:
import base64
import io
import json
import logging
import boto3
from PIL import Image
from botocore.config import Config
from botocore.exceptions import ClientError

In [113]:
def generate_image(body):
    """
    Generate an image using Amazon Nova Canvas model on demand.
    Args:
        body (str) : The request body to use.
    Returns:
        image_bytes (bytes): The image generated by the model.
    """

    bedrock = boto3.client(
        service_name='bedrock-runtime',
        region_name='us-east-1',
        config=Config(read_timeout=300)
    )

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId='amazon.nova-canvas-v1:0', accept=accept, contentType=content_type
    )
    response_body = json.loads(response.get("body").read())

    base64_image = response_body.get("images")[0]
    base64_bytes = base64_image.encode('ascii')
    image_bytes = base64.b64decode(base64_bytes)

    finish_reason = response_body.get("error")

    if finish_reason is not None:
        raise ImageError(f"Image generation error. Error is {finish_reason}")

    return image_bytes

In [114]:
body = json.dumps({
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": generated_prompt,
        "negativeText":negative_prompt
    },
    "imageGenerationConfig": {
        "numberOfImages": 2,
        "height": 1280,
        "width": 720,
        "cfgScale": 8.0,
        "seed": 0
    }
})


In [ ]:
try:
    image_bytes = generate_image( body=body)
    image = Image.open(io.BytesIO(image_bytes))
    image.show()

except ClientError as err:
    message = err.response["Error"]["Message"]
    print("A client error occured: " +
          format(message))
except Exception as err:
    print(err.message)
